In [1]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "7"
sys.path.append('../') 

In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

import random
from demo.ConfSeq import get_ConfSeq_pair_from_mol,get_mol_from_ConfSeq_pair,randomize_mol
import copy
from tqdm import tqdm
from rdkit.Chem import rdmolops, rdchem
from tqdm.contrib.concurrent import process_map  

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from transformers import BartForConditionalGeneration, BartConfig
from collections import OrderedDict
import torch
from torch import nn

import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

from rdkit.ML.Scoring import Scoring
import pandas as pd

/home/xiongjiacheng/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
vocab = [chr(i) for i in range(33, 127)] 

for i in range(-180,180):
    vocab.append('<'+str(i)+'>')

vocab.append('<mask>')
vocab.append('<unk>')
vocab.append('<sos>')
vocab.append('<eos>')
vocab.append('<pad>')


config = BartConfig()

config.pad_token_id = vocab.index('<pad>')
config.eos_token_id = vocab.index('<eos>')
config.sos_token_id = vocab.index('<sos>')
config.forced_eos_token_id = None
config.encoder_layers = 6
config.encoder_attention_heads = 8
config.decoder_layers = 0
config.decoder_attention_heads = 0
config.d_model = 256
# config.share_embeddings = True
config.vocab_size = len(vocab)


vocab_dict = {char: idx for idx, char in enumerate(vocab)}
bart = BartForConditionalGeneration(config = config )


class CustomBartEncoder(nn.Module):
    def __init__(self, bart):
        super().__init__()
    
        # 加载 BART 模型
        self.bart_model = bart 
        
    def forward(self, input_ids, attention_mask=None):
        # 获取编码器输出
        outputs = self.bart_model.model.encoder(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state

# 示例使用
bart_encoder_model  = CustomBartEncoder(bart)

In [4]:
def rm_invalid_chirality(mol):
    mol = copy.deepcopy(mol)
    """
    找出分子中同时出现在三个环中的原子。
    
    参数:
        mol: RDKit 分子对象
    返回:
        List[int]: 同时出现在三个环中的原子的索引列表
    """
    # 获取分子的所有环（SSSR：最小集的简单环）
    rings = rdmolops.GetSymmSSSR(mol)

    # 创建一个字典，记录每个原子出现在多少个环中
    atom_in_rings_count = {}

    # 遍历所有环，统计每个原子出现的次数
    for ring in rings:
        for atom_idx in ring:
            if atom_idx not in atom_in_rings_count:
                atom_in_rings_count[atom_idx] = 0
            atom_in_rings_count[atom_idx] += 1

    # 找出那些同时出现在三个环中的原子
    atoms_in_3_rings = [atom for atom, count in atom_in_rings_count.items() if count == 3]

    for atom_idx in atoms_in_3_rings:
        atom = mol.GetAtomWithIdx(atom_idx)
        atom.SetChiralTag(rdchem.ChiralType.CHI_UNSPECIFIED)

    return mol

In [5]:
def get_ConfSeq(query_mol):

    if query_mol != None:
        try:
            query_mol = rm_invalid_chirality(query_mol)
            query_mol = randomize_mol(query_mol)
            Chem.MolToSmiles(query_mol)    
            query_mol = Chem.RenumberAtoms(query_mol, eval(query_mol.GetProp('_smilesAtomOutputOrder'))) 
            Chem.MolToSmiles(query_mol,canonical = False)
            query_mol = Chem.RenumberAtoms(query_mol, eval(query_mol.GetProp('_smilesAtomOutputOrder'))) 
            in_smiles,TD_smiles = get_ConfSeq_pair_from_mol(query_mol)
            TD_smiles = TD_smiles.replace('<180>','<-180>')
            
        except:
            in_smiles,TD_smiles = '',''            
    else:
        in_smiles,TD_smiles = '',''
        
    return TD_smiles

In [6]:
def get_embedding_for_seq(i):
    int = [vocab_dict[i] for i in i.split(' ')] 
    int = torch.Tensor([int]).long()
    embed = bart_encoder_model(int).mean(dim=1).tolist()[0]
    return embed

In [7]:
class TensorDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.Tensor(self.data[idx]).long()

def collate_fn(batch):
    data = batch  # 解压每个样本中的数据和标签
    padded_data = pad_sequence(data, batch_first=True, padding_value = vocab.index('<pad>'))  # 对数据进行填充
    return padded_data


def mean_pooling(last_hidden_state, attention_mask):
    # 对每个样本进行池化，忽略 pad 的位置
    # 将 attention_mask 转换为 float 类型，并进行扩展
    attention_mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size())
    # 计算有效的特征总和和有效特征的数量
    sum_embeddings = (last_hidden_state * attention_mask).sum(dim=1)
    sum_mask = attention_mask.sum(dim=1)
    # 计算平均池化，避免除以零
    pooled_output = sum_embeddings / (sum_mask + 1e-8)  # 加小常数以避免除零
    return pooled_output

In [8]:
device = torch.device('cuda')

In [9]:
checkpoint = torch.load('./checkpoints/model_epoch_1.pth', map_location='cpu')  # 使用适当的设备    

new_state_dict = OrderedDict()
for k, v in checkpoint.items():
    if k.startswith('module.'):
        new_state_dict[k[7:]] = v
    else:
        new_state_dict[k] = v

bart_encoder_model.load_state_dict(new_state_dict)
bart_encoder_model.eval()


lis = os.listdir('./data/PCBA/AVE_unbiased/')
lis = [i for i in lis if i[-4:] != '.txt']

for name in lis[:]:

    os.makedirs('./data/PCBA/AVE_unbiased_result/{}/'.format(name), exist_ok=True)
    
    file_lis = os.listdir('./data/PCBA/AVE_unbiased/{}/'.format(name))
    file_lis = [i for i in file_lis if 'ligand_.sdf' in i]

    suppl = Chem.SDMolSupplier('./data/PCBA/AVE_unbiased/{}/actives.sdf'.format(name))
    # 将有效的分子（非 None）保存在列表中
    actives_mols = [mol for mol in suppl if mol is not None]
    actives_mols_atom_num = [i.GetNumAtoms() for i in actives_mols]
    
    suppl = Chem.SDMolSupplier('./data/PCBA/AVE_unbiased/{}/inactives.sdf'.format(name))
    # 将有效的分子（非 None）保存在列表中
    decoys_mols = [mol for mol in suppl if mol is not None]
    decoys_mols_atom_num = [i.GetNumAtoms() for i in decoys_mols]


    actives_seqs = process_map(get_ConfSeq, tqdm(actives_mols), max_workers = 32)
    print('actives_seqs length:',len(actives_seqs))
    
    decoys_seqs = process_map(get_ConfSeq, tqdm(decoys_mols), max_workers = 32)
    print('decoys_seqs length:',len(decoys_seqs))

    with open('./data/PCBA/AVE_unbiased_result/{}/actives_seqs.txt'.format(name),'w+') as f:
        f.write('\n'.join(actives_seqs))

    with open('./data/PCBA/AVE_unbiased_result/{}/decoys_seqs.txt'.format(name),'w+') as f:
        f.write('\n'.join(decoys_seqs))


    with open('./data/PCBA/AVE_unbiased/{}/actives_seqs.txt'.format(name),'r') as f:
        content = f.read()
    active_seqs = content.split('\n')   
    active_seqs = [i if i != '' else '*' for i in actives_seqs]    
    
    with open('./data/PCBA/AVE_unbiased/{}/decoys_seqs.txt'.format(name),'r') as f:
        content = f.read()
    decoys_seqs = content.split('\n') 
    decoys_seqs = [i if i != '' else '*' for i in decoys_seqs]


    active_ints = [[vocab_dict[i] for i in seq.split(' ')] for seq in actives_seqs]
    active_dataset = TensorDataset(active_ints)
    active_dataloader = DataLoader(active_dataset, batch_size=128,collate_fn=collate_fn)

    bart_encoder_model.to(device)
    
    active_embeds = []
    
    with torch.no_grad():  # 取消梯度计算
    
        for input_ids in active_dataloader:
            input_ids = input_ids.to(device)
            attention_mask = (input_ids != vocab.index('<pad>')).long() # 创建 attention mask
            # 编码
            output = bart_encoder_model(input_ids,attention_mask)
            pooled_output = mean_pooling(output, attention_mask)
            active_embeds.append(pooled_output)
    
    active_embeds = torch.cat(tuple(active_embeds), dim=0).to('cpu')

    inactive_ints = [[vocab_dict[i] for i in seq.split(' ')] for seq in decoys_seqs]
    inactive_dataset = TensorDataset(inactive_ints)
    inactive_dataloader = DataLoader(inactive_dataset, batch_size=128,collate_fn=collate_fn)

    
    inactive_embeds = []
    
    with torch.no_grad():  # 取消梯度计算
    
        for input_ids in inactive_dataloader:
            input_ids = input_ids.to(device)
            attention_mask = (input_ids != vocab.index('<pad>')).long() # 创建 attention mask
            # 编码
            output = bart_encoder_model(input_ids,attention_mask)
            pooled_output = mean_pooling(output, attention_mask)
            inactive_embeds.append(pooled_output)
    
    inactive_embeds = torch.cat(tuple(inactive_embeds), dim=0).to('cpu')

    
    fingerprints = np.vstack([active_embeds,inactive_embeds])
    labels = np.array([1] * len(active_embeds) + [0] * len(inactive_embeds))  # 活性为1，非活性为0   

    mols_atom_num = np.array(actives_mols_atom_num + decoys_mols_atom_num)
    
    for file in file_lis:    
        print(file)

        query_mol = Chem.MolFromMolFile('./data/PCBA/AVE_unbiased/{}/{}'.format(name,file))
        query_mol_atom_num = query_mol.GetNumAtoms()
        
        bart_encoder_model.to(torch.device('cpu'))
        query_embeds = get_embedding_for_seq(get_ConfSeq(query_mol))
        query_embeds = torch.Tensor(query_embeds)
        euclidean_distance = torch.sqrt(torch.sum((query_embeds - fingerprints ) ** 2,dim = -1))
        similarity_score = 1 / (1 + euclidean_distance)

        factor = 2 * mols_atom_num/ (query_mol_atom_num + mols_atom_num)
        similarity_score = similarity_score * factor
        
        fpr, tpr, thresholds = roc_curve(labels.tolist(), similarity_score.tolist())
        
        # 计算 AUC
        roc_auc = auc(fpr, tpr)

        similarity_score_np = similarity_score.numpy()
        sort_idx = np.argsort(-similarity_score_np)
        sorted_scores = similarity_score_np[sort_idx]
        sorted_labels = labels[sort_idx].astype(bool)
        score_data = np.column_stack((sorted_scores, sorted_labels))

        bedroc_rdkit = Scoring.CalcBEDROC(score_data, col=1, alpha=80.5)
        ef_1 = Scoring.CalcEnrichment(score_data, col=1, fractions=[0.01])[0]
        ef_5 = Scoring.CalcEnrichment(score_data, col=1, fractions=[0.05])[0]

        # 构造结果行
        row = {
            "Protein": name,
            "Ligand_File": file,
            "ROC_AUC": round(roc_auc, 4),
            "BEDROC": round(bedroc_rdkit, 4),
            "EF5%": round(ef_5, 4),
            "EF1%": round(ef_1, 4)
        }

        # 写入 CSV 文件：如果文件不存在，则写入表头，否则追加行
        csv_path = f'./PCBA_evaluation_result.csv'
        if not os.path.exists(csv_path):
            df = pd.DataFrame([row])
            df.to_csv(csv_path, index=False)
        else:
            df = pd.DataFrame([row])
            df.to_csv(csv_path, mode='a', index=False, header=False)

100%|████████████████████████████████████████████| 39/39 [00:00<00:00, 801.68it/s]


actives_seqs length: 39


  0%|                                                  | 0/358579 [00:00<?, ?it/s]/tmp/ipykernel_2676189/438043969.py:38: TqdmWarning: Iterable length 358579 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  decoys_seqs = process_map(get_ConfSeq, tqdm(decoys_mols), max_workers = 32)
100%|███████████████████████████████████| 358579/358579 [02:17<00:00, 2603.21it/s]


decoys_seqs length: 358579
5lge_ligand_.sdf
4xs3_ligand_.sdf
5de1_ligand_.sdf
5l57_ligand_.sdf
4i3l_ligand_.sdf
4umx_ligand_.sdf
4xrx_ligand_.sdf
6adg_ligand_.sdf
6b0z_ligand_.sdf
5sun_ligand_.sdf
5l58_ligand_.sdf
4i3k_ligand_.sdf
5svf_ligand_.sdf
5tqh_ligand_.sdf


100%|████████████████████████████████████████████| 24/24 [00:00<00:00, 457.54it/s]


...

decoys_seqs length: 3818
3dt3_ligand_.sdf
6b0f_ligand_.sdf
6chw_ligand_.sdf
2iog_ligand_.sdf
2r6w_ligand_.sdf
5fqv_ligand_.sdf
2pog_ligand_.sdf
1xp1_ligand_.sdf
2ayr_ligand_.sdf
1xqc_ligand_.sdf
2iok_ligand_.sdf
5t92_ligand_.sdf
5ufx_ligand_.sdf
5aau_ligand_.sdf
2ouz_ligand_.sdf


100%|█████████████████████████████████████████| 194/194 [00:00<00:00, 1108.75it/s]


actives_seqs length: 194


  0%|                                                  | 0/342518 [00:00<?, ?it/s]/tmp/ipykernel_2676189/438043969.py:38: TqdmWarning: Iterable length 342518 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  decoys_seqs = process_map(get_ConfSeq, tqdm(decoys_mols), max_workers = 32)
100%|███████████████████████████████████| 342518/342518 [01:57<00:00, 2926.77it/s]


decoys_seqs length: 342518
5h84_ligand_.sdf
5h86_ligand_.sdf
5mlj_ligand_.sdf
